In [1]:
import time, json, math, threading
from pathlib import Path

import numpy as np
import PIL.Image

import torch
import torchvision
import torchvision.transforms as transforms

import paho.mqtt.client as mqtt
from jetbot import Robot, Camera

import Adafruit_SSD1306
import os, time
import json
import subprocess
from jetbot.utils.utils import get_ip_address
from jetbot import ina219 
from jetbot import ads1115

# ===== MQTT 설정 (기존 IP 유지) =====
BROKER_IP    = "10.41.145.221"
BROKER_PORT  = 1883
TOPIC_TASK   = "robot/agv1/task"
TOPIC_STATUS = "robot/agv1/status"

CLIENT_ID = f"jetbot-agv1-step3-{int(time.time()*1000)}"
robot_id = "agv1"

# ===== 파일 경로 =====
MODEL_PATH = Path("pjt/models/best_steering_model_xy_test.pth")
TRACK_PATH = Path("track_agv1.json")

# ===== 타이밍 설정 =====
STATUS_PERIOD_SEC = 0.5
LINE_LOOP_DT = 0.02

# ===== 라인 트레이싱 파라미터 =====
SPEED = 0.30
STEERING_GAIN  = 0.20
STEERING_DGAIN = 0.05
STEERING_BIAS  = 0.00

LOG_EVERY_SEC = 1.0
PRINT_MOTOR   = True

# ===== 디지털 트윈 랩타임 (보정용) =====
LAP_TIME_SEC = 56.22

# ===== UserApp 기준 좌표 (변경 없음) =====
AREA_POS_UI = {
    "CHARGE": {"x": 50, "y": 10},
    "WATER":  {"x": 18, "y": 28},
    "DROP":   {"x": 82, "y": 42},
    "RES_A":  {"x": 20, "y": 70},
    "RES_B":  {"x": 42, "y": 82},
    "RES_C":  {"x": 70, "y": 82},
}

print("[OK] Cell 1 Loaded")

[OK] Cell 1 Loaded


In [2]:
# ===== [Step 4] 객체 인식(OD) 모델 로딩 =====
from tflite_runtime.interpreter import Interpreter
import cv2
import numpy as np

# 팀원 코드 경로: models/best-fp16.tflite
# Step3 경로 기준: pjt/models/best-fp16.tflite 라고 가정 (안 되면 경로 수정 필요)
OD_MODEL_PATH = Path("pjt/models/best-fp16.tflite") 

# 경로 안전 장치
if not OD_MODEL_PATH.exists():
    # 혹시 몰라 현재 경로 기준도 체크
    OD_MODEL_PATH = Path("models/best-fp16.tflite")

print(f"[OD] Loading model from: {OD_MODEL_PATH}")

try:
    od_interpreter = Interpreter(model_path=str(OD_MODEL_PATH))
    od_interpreter.allocate_tensors()
    od_input_details = od_interpreter.get_input_details()
    od_output_details = od_interpreter.get_output_details()
    
    OD_LABELS = ['block', 'cup', 'doll'] # 팀원 코드 기준 라벨
    
    print("[OD] Model Loaded Successfully ✅")
    
except Exception as e:
    print(f"[OD] ❌ Model Load Error: {e}")
    od_interpreter = None

# 팀원 코드의 get_usb_dets를 가져와서 JetBot 카메라 포맷(frame)에 맞게 수정
def detect_objects(frame):
    if od_interpreter is None or frame is None:
        return []
        
    h, w = frame.shape[:2]
    # 입력 전처리: 224x224 리사이즈 -> RGB 변환 -> Normalize
    img = cv2.resize(frame, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    
    # 추론 실행
    od_interpreter.set_tensor(od_input_details[0]['index'], np.expand_dims(img, axis=0))
    od_interpreter.invoke()
    output = od_interpreter.get_tensor(od_output_details[0]['index'])
    
    # 후처리 (NMS)
    boxes, scores, classes = [], [], []
    for row in output[0]:
        if row[4] > 0.5: # Confidence Threshold (팀원 코드는 0.6이었으나 0.5로 살짝 완화)
            cls_id = np.argmax(row[5:])
            cx, cy, bw, bh = row[:4]
            boxes.append([int((cx-bw/2)*w), int((cy-bh/2)*h), int((cx+bw/2)*w), int((cy+bh/2)*h)])
            scores.append(float(row[4] * row[5+cls_id]))
            classes.append(int(cls_id))
    
    # NMS 적용
    if len(boxes) > 0:
        idxs = cv2.dnn.NMSBoxes(boxes, scores, 0.35, 0.45)
        if len(idxs) > 0:
            results = []
            for i in idxs.flatten():
                lbl = OD_LABELS[classes[i]]
                results.append((lbl, scores[i], boxes[i]))
            return results
            
    return []

[OD] Loading model from: models/best-fp16.tflite
[OD] Model Loaded Successfully ✅


In [3]:
def load_track_points(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"track json not found: {path.resolve()}")

    obj = json.loads(path.read_text(encoding="utf-8"))

    if isinstance(obj, dict) and isinstance(obj.get("dense"), list):
        pts = obj["dense"]
    elif isinstance(obj, list):
        pts = obj
    else:
        raise ValueError("Unknown track json format")

    out = []
    for p in pts:
        if not isinstance(p, dict): continue
        x, y = p.get("x"), p.get("y")
        if x is None or y is None: continue
        if not np.isfinite(x) or not np.isfinite(y): continue
        out.append({"x": float(x), "y": float(y)})

    # 트랙 닫힘 처리 (시작점==끝점이면 중복 제거)
    if len(out) >= 2:
        dx = out[0]["x"] - out[-1]["x"]
        dy = out[0]["y"] - out[-1]["y"]
        if dx*dx + dy*dy < 1e-9:
            out = out[:-1]

    if len(out) < 2:
        raise ValueError("track points too short")

    return out

TRACK_POINTS = load_track_points(TRACK_PATH)
TRACK_N = len(TRACK_POINTS)
POINTS_PER_SEC = TRACK_N / max(LAP_TIME_SEC, 1e-6)

def nearest_track_idx(x, y):
    best_i, best_d = 0, 1e18
    for i, p in enumerate(TRACK_POINTS):
        dx = p["x"] - x
        dy = p["y"] - y
        d = dx*dx + dy*dy
        if d < best_d:
            best_d, best_i = d, i
    return best_i

TRACK_START_IDX = nearest_track_idx(AREA_POS_UI["CHARGE"]["x"], AREA_POS_UI["CHARGE"]["y"])

print("[TRACK] N =", TRACK_N)
print("[TRACK] START_IDX =", TRACK_START_IDX)

# AREA 좌표 -> 트랙 인덱스 매핑
AREA_TO_IDX = {k: nearest_track_idx(v["x"], v["y"]) for k, v in AREA_POS_UI.items()}

print("[AREA_TO_IDX]")
for k in AREA_TO_IDX:
    print(f" {k} -> {AREA_TO_IDX[k]}")

[TRACK] N = 800
[TRACK] START_IDX = 794
[AREA_TO_IDX]
 CHARGE -> 794
 WATER -> 134
 DROP -> 615
 RES_A -> 259
 RES_B -> 367
 RES_C -> 452


In [4]:
import os
from pathlib import Path

def resolve_path(p: Path) -> Path:
    if p.exists():
        return p.resolve()
    # 경로 못 찾을 시 후보군 탐색
    candidates = [
        Path.cwd() / p,
        Path.cwd() / "pjt" / "models" / p.name,
        Path.cwd().parent / "pjt" / "models" / p.name,
        Path("/home/jetbot/Notebooks") / p,
    ]
    for c in candidates:
        if c.exists(): return c.resolve()
    
    # 최후의 수단: 전체 검색
    matches = list(Path.cwd().rglob(p.name))
    if matches: return matches[0].resolve()
    
    raise FileNotFoundError(f"NOT FOUND: {p}")

print("[CWD]", Path.cwd())

MODEL_PATH = resolve_path(MODEL_PATH)
TRACK_PATH = resolve_path(TRACK_PATH)

print("[OK] MODEL_PATH =", MODEL_PATH)
print("[OK] TRACK_PATH =", TRACK_PATH)

[CWD] /workspace/pjt
[OK] MODEL_PATH = /workspace/pjt/models/best_steering_model_xy_test.pth
[OK] TRACK_PATH = /workspace/pjt/track_agv1.json


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_fp16 = (device.type == "cuda")

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

state_dict = torch.load(str(MODEL_PATH), map_location="cpu")
model.load_state_dict(state_dict)

model = model.to(device).eval()
if use_fp16:
    model = model.half()

mean = torch.tensor([0.485, 0.456, 0.406], device=device)
std  = torch.tensor([0.229, 0.224, 0.225], device=device)
if use_fp16:
    mean = mean.half()
    std  = std.half()

def preprocess(frame_bgr_or_rgb):
    img = PIL.Image.fromarray(frame_bgr_or_rgb)
    t = transforms.functional.to_tensor(img).to(device)
    if use_fp16:
        t = t.half()
    t.sub_(mean[:, None, None]).div_(std[:, None, None])
    return t[None, ...]

print(f"[OK] model loaded | device={device} fp16={use_fp16}")

[OK] model loaded | device=cuda fp16=True


In [6]:
try: robot.stop()
except: pass

robot = Robot()
robot.stop()
camera = Camera()

print("[OK] Robot & Camera ready")

def prime_motors(val=0.15, duration=0.25):
    robot.left_motor.value  = val
    robot.right_motor.value = val
    time.sleep(duration)
    robot.stop()
    time.sleep(0.15)

prime_motors()
print("[OK] motors primed")

@torch.no_grad()
def warmup_model(n=20):
    for _ in range(n):
        f = camera.value
        if f is None:
            time.sleep(0.05); continue
        _ = model(preprocess(f))
        time.sleep(0.02)

warmup_model(20)
print("[OK] model warmup done")

# ===== [추가] 배터리 센서 (INA219) 초기화 =====
ina = None
try:
    ina = ina219.INA219(addr=0x41)
    print("[OK] INA219 Battery Sensor Connected")
except Exception as e:
    print(f"[WARN] INA219 sensor not found: {e}")
    ina = None

[OK] Robot & Camera ready
[OK] motors primed
[OK] model warmup done
[OK] INA219 Battery Sensor Connected


In [7]:
# ----- Shared State -----
state_lock = threading.Lock()

state = "idle"            # idle / running / working / error / done
current_task_id = None
area = "CHARGE"
target_area = None
error_code = None
battery = 66

# ----- Step3 핵심: Waypoints Queue -----
current_waypoints = []   # 예: ["WATER", "RES_A", "CHARGE"]
target_idx = None        # 현재 목적지 인덱스
track_dir = +1           # +1 정방향, -1 역방향

# ----- Timeouts -----
done_until = 0.0
ARRIVE_IDX_TOL = 4
ARRIVE_HOLD_SEC = 0.4
DONE_HOLD_SEC = 1.0
WORKING_SEC = 2.0        # 도착 후 가짜 작업 시간 (2초)

# ----- Track Progress (Digital Twin) -----
track_lock = threading.Lock()
track_idx_f = float(TRACK_START_IDX)
track_last_t = time.monotonic()

def reset_track_to_start():
    global track_idx_f, track_last_t
    with track_lock:
        track_idx_f = float(TRACK_START_IDX)
        track_last_t = time.monotonic()

def update_track_progress_if_running(is_running: bool, did_drive: bool):
    global track_idx_f, track_last_t, track_dir
    now = time.monotonic()
    with track_lock:
        dt = now - track_last_t
        track_last_t = now
        if dt < 0 or dt > 0.2: return
        
        # 실제로 모터가 돌 때만 트랙 인덱스 이동
        if not (is_running and did_drive):
            return

        track_idx_f = (track_idx_f + dt * POINTS_PER_SEC * track_dir) % TRACK_N

def get_track_pose():
    with track_lock:
        idx = int(track_idx_f) % TRACK_N
    step = 1 if track_dir >= 0 else -1
    p = TRACK_POINTS[idx]
    p2 = TRACK_POINTS[(idx + step) % TRACK_N]
    theta = math.atan2(p2["y"] - p["y"], p2["x"] - p["x"])
    return {"x": float(p["x"]), "y": float(p["y"]), "theta": float(theta)}, {"track_idx": idx, "dir": track_dir}

def choose_shortest_dir(cur_idx: int, tgt_idx: int) -> int:
    fwd = (tgt_idx - cur_idx) % TRACK_N
    bwd = (cur_idx - tgt_idx) % TRACK_N
    return +1 if fwd <= bwd else -1

def ring_dist(a: int, b: int) -> int:
    d1 = (b - a) % TRACK_N
    d2 = (a - b) % TRACK_N
    return min(d1, d2)

print("[OK] Shared State & Waypoints Ready")

[OK] Shared State & Waypoints Ready


In [ ]:
# ===== [Step 4 Final: Resource Optimization] USB 카메라 On-Demand 방식 =====
import threading
import time
import numpy as np
import cv2
from SCSCtrl import TTLServo
from tflite_runtime.interpreter import Interpreter

# =========================================================
# 0. 헬퍼 함수 & 안전장치
# =========================================================
if 'track_lock' not in globals():
    track_lock = threading.Lock()
    track_idx_f = 0.0
    track_last_t = time.monotonic()
    track_dir = 1

def update_track_progress_if_running(is_running: bool, did_drive: bool):
    global track_idx_f, track_last_t, track_dir
    _PPS = globals().get('POINTS_PER_SEC', 10.0) 
    _TN = globals().get('TRACK_N', 800)
    now = time.monotonic()
    with track_lock:
        dt = now - track_last_t
        track_last_t = now
        if dt < 0 or dt > 0.2: return
        if not (is_running and did_drive): return
        track_idx_f = (track_idx_f + dt * _PPS * track_dir) % _TN

def choose_shortest_dir(cur_idx: int, tgt_idx: int) -> int:
    _TN = globals().get('TRACK_N', 800)
    fwd = (tgt_idx - cur_idx) % _TN
    bwd = (cur_idx - tgt_idx) % _TN
    return +1 if fwd <= bwd else -1

def ring_dist(a: int, b: int) -> int:
    _TN = globals().get('TRACK_N', 800)
    d1 = (b - a) % _TN
    d2 = (a - b) % _TN
    return min(d1, d2)

def snap_track_idx(idx: int):
    global track_idx_f, track_last_t
    with track_lock:
        track_idx_f = float(idx)
        track_last_t = time.monotonic()

# ★ 객체 인식 함수 (라벨 하드코딩)
def detect_objects_debug(frame):
    if 'od_interpreter' not in globals() or od_interpreter is None:
        return []
    if frame is None: return []

    MY_LABELS = ['block', 'cup', 'doll']

    h, w = frame.shape[:2]
    img = cv2.resize(frame, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    od_interpreter.set_tensor(od_input_details[0]['index'], np.expand_dims(img, axis=0))
    od_interpreter.invoke()
    output = od_interpreter.get_tensor(od_output_details[0]['index'])

    boxes, scores, classes = [], [], []
    for row in output[0]:
        score = float(row[4])
        if score > 0.4:  
            cls_id = int(np.argmax(row[5:]))
            cx, cy, bw, bh = row[:4]
            boxes.append([int((cx-bw/2)*w), int((cy-bh/2)*h), int((cx+bw/2)*w), int((cy+bh/2)*h)])
            scores.append(float(score * row[5+cls_id]))
            if cls_id < len(MY_LABELS): classes.append(MY_LABELS[cls_id])
            else: classes.append(str(cls_id))

    if len(boxes) > 0:
        idxs = cv2.dnn.NMSBoxes(boxes, scores, 0.35, 0.45)
        if len(idxs) > 0:
            results = []
            for i in idxs.flatten():
                results.append((classes[i], scores[i], boxes[i]))
            return results
    return []

# =========================================================
# 1. 전역 변수 설정
# =========================================================
line_stop = threading.Event()
line_follow_enabled = False
_lock = threading.Lock()

_PRINT_MOTOR = globals().get('PRINT_MOTOR', True)
_LOG_EVERY_SEC = globals().get('LOG_EVERY_SEC', 1.0)
_last_log_t = 0.0

_ARRIVE_IDX_TOL = globals().get('ARRIVE_IDX_TOL', 4)
_ARRIVE_HOLD_SEC = globals().get('ARRIVE_HOLD_SEC', 0.4)
_DONE_HOLD_SEC = globals().get('DONE_HOLD_SEC', 1.0) # Done 유지 시간

force_arrival = False 
current_task_type = "unknown"
angle_last = 0.0
STOP_LINE_Y_CUP = 110 
CENTER_Y = 112 

# =========================================================
# 2. 하드웨어 클래스 (On-Demand Camera)
# =========================================================
class AgvHardware:
    def __init__(self):
        self.pan_angle = 5
        self.servo1_speed = 0.05
        self.forward_speed = 0.13
        
        # 좌표값
        self.CUP_APPROACH = (130, -140)
        self.CUP_GRAB     = (180, -125)
        self.CUP_LIFT     = (180, -100)
        self.CUP_PLACE    = (180, -130)
        self.DOLL_APPROACH = (150, -140)
        self.DOLL_GRAB     = (150, -140)
        self.DOLL_LIFT     = (150, 80)
        self.DOLL_PLACE    = (150, -110)
        
        # 초기화 시엔 카메라는 안 검. 서보만 리셋.
        self.reset_servos()

    # ★ [핵심] 필요할 때만 열고 닫는 함수
    def open_usb_camera(self):
        print("[HW] Opening USB Camera...")
        cap = cv2.VideoCapture(1)
        if not cap.isOpened():
            print("[HW] ⚠️ Video(1) failed. Trying Video(0)...")
            cap = cv2.VideoCapture(0)
        return cap

    def reset_servos(self):
        try:
            TTLServo.servoAngleCtrl(4, 5, 1, 150)
            TTLServo.servoAngleCtrl(5, 50, 1, 100) # 주행 각도
            TTLServo.servoAngleCtrl(3, 20, 1, 150)
            time.sleep(0.5)
            TTLServo.xyInput(100, 100)
            time.sleep(0.5)
            TTLServo.servoAngleCtrl(1, 5, 1, 100)
            self.pan_angle = 5
        except: pass

    # --- Motions ---
    def _pick_cup_motion(self):
        print("🦾 [HW] Picking Cup...")
        TTLServo.servoAngleCtrl(4, 10, 1, 150) 
        TTLServo.xyInputSmooth(130, -140, 3.0); time.sleep(4.0) 
        TTLServo.xyInputSmooth(180, -125, 3.0); time.sleep(3.0) 
        TTLServo.servoAngleCtrl(4, -25, 1, 150); time.sleep(2.0) 
        TTLServo.xyInputSmooth(180, -100, 3.0); time.sleep(3.0) 

    def _place_cup_motion(self):
        print("🦾 [HW] Placing Cup...")
        TTLServo.xyInputSmooth(180, -130, 3.0); time.sleep(3.0)
        TTLServo.servoAngleCtrl(4, 10, 1, 150); time.sleep(2.0)
        self.reset_servos()

    def _pick_doll(self):
        print("🦾 [HW] Action: Picking Doll")
        TTLServo.servoAngleCtrl(4, 10, 1, 150) 
        TTLServo.xyInput(150, -140); time.sleep(2.0)
        TTLServo.servoAngleCtrl(4, -80, 1, 150); time.sleep(2.0)
        TTLServo.xyInput(150, 80); time.sleep(3.0)
    
    def _place_doll(self):
        print("🦾 [HW] Action: Placing Doll")
        TTLServo.xyInput(150, -110); time.sleep(1.0)
        TTLServo.servoAngleCtrl(4, 10, 1, 150); time.sleep(2.0)

    def _pick_block(self):
        print("🦾 [HW] Action: Picking Block")
        TTLServo.servoAngleCtrl(4, 10, 1, 150) 
        TTLServo.xyInputSmooth(150, -125, 2.0); time.sleep(2.0)
        TTLServo.servoAngleCtrl(4, -30, 1, 150); time.sleep(0.5)
        TTLServo.xyInput(150, 80); time.sleep(3.0)

    def _place_block(self):
        print("🦾 [HW] Action: Placing Block")
        TTLServo.xyInput(150, -120); time.sleep(1.0)
        TTLServo.servoAngleCtrl(3, -5, 1, 150); time.sleep(0.5)
        TTLServo.servoAngleCtrl(4, 10, 1, 150); time.sleep(2.0)

    # --- Align & Pick (카메라 열고 -> 작업 -> 닫고) ---
    def align_and_pick(self, target_label):
        # 1. 카메라 열기
        cap = self.open_usb_camera()
        if not cap.isOpened():
            print("[HW] ❌ Camera Open Failed. Skip job.")
            return

        print(f"👀 [ALIGN] Searching for '{target_label}'...")
        TTLServo.servoAngleCtrl(5, 50, 1, 100) 
        time.sleep(1.0)
        
        t_start = time.time()
        step = 1 
        last_seen_time = time.time()
        final_label = "unknown"
        
        try:
            while time.time() - last_seen_time < 15.0:
                robot.stop()
                time.sleep(0.1) 
                
                ret, frame = cap.read()
                if not ret: 
                    print("!", end="", flush=True); time.sleep(0.1); continue
                
                h, w = frame.shape[:2]
                real_center_x = w // 2
                if target_label == 'cup': real_stop_y = int(h * 0.23) 
                else: real_stop_y = h // 2

                dets = detect_objects_debug(frame)
                target_det = None

                if target_request == 'laundry':
                    # laundry는 'doll' 또는 'block' 중 먼저 발견된 것을 타겟으로 함
                    for d in dets:
                        if 'doll' in d[0] or 'block' in d[0]:
                            target_det = d; break
                else: # 'cup' 등 단일 타겟
                    for d in dets:
                        if target_request in d[0]:
                            target_det = d; break
                
                if not target_det:
                    print(".", end="", flush=True) 
                    continue
                
                last_seen_time = time.time()
                print(f" FOUND! {target_det}")
                (label, score, box) = target_det
                final_label = label
                x1, y1, x2, y2 = box
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                
                if step == 1:
                    target_y = real_stop_y
                    offset_y = cy - target_y
                    if abs(offset_y) > 5: 
                        speed = self.forward_speed if offset_y < 0 else -self.forward_speed
                        robot.left_motor.value = speed; robot.right_motor.value = speed
                        time.sleep(0.15); robot.stop()
                    else:
                        robot.stop()
                        if 'cup' in label:
                             robot.left_motor.value = -0.13; robot.right_motor.value = -0.13
                             time.sleep(0.2); robot.stop()
                        step = 2
                        print(f"\n✅ [ALIGN] Y-Align Done.")
                        time.sleep(0.5)
                elif step == 2:
                    offset_x = cx - real_center_x
                    if abs(offset_x) > 5:
                        self.pan_angle += offset_x * self.servo1_speed
                        self.pan_angle = np.clip(self.pan_angle, -80, 80)
                        TTLServo.servoAngleCtrl(1, int(self.pan_angle), 1, 150)
                        time.sleep(0.2) 
                    else:
                        print(f"🎯 [ALIGN] X-Align Done. PICKING!")
                        robot.stop()
                        time.sleep(0.5)
                        
                        # 카메라 끄고 집기
                        cap.release()

                        # [최종 동작 분기]
                        if 'cup' in final_label:
                            self._pick_cup_motion()
                        elif 'doll' in final_label:
                            self._pick_doll()
                        elif 'block' in final_label:
                            self._pick_block()
                        return
                        
            print("\n⚠️ [ALIGN] Timeout.")
            
        finally:
            # 작업 끝나거나 에러나면 무조건 카메라 닫기
            if cap.isOpened(): cap.release()
            print("[HW] USB Camera Released.")
            
        robot.stop()
        self.reset_servos()

agv_hw = AgvHardware()

# =========================================================
# 3. 헬퍼 함수
# =========================================================
def flush_camera(n=5):
    for _ in range(n):
        _ = camera.value
        time.sleep(0.01)

def smart_turn_180(direction="left"):
    print("[TURN] Smart 180 started...")
    BLIND_SEC = 4.5; BLIND_POWER = 0.4
    if direction == "left":
        robot.left_motor.value  = -BLIND_POWER; robot.right_motor.value = +BLIND_POWER
    else:
        robot.left_motor.value  = +BLIND_POWER; robot.right_motor.value = -BLIND_POWER
    time.sleep(BLIND_SEC)
    
    t0 = time.time()
    SEARCH_POWER = 0.15
    if direction == "left":
        robot.left_motor.value  = -SEARCH_POWER; robot.right_motor.value = +SEARCH_POWER
    else:
        robot.left_motor.value  = +SEARCH_POWER; robot.right_motor.value = -SEARCH_POWER
    while time.time() - t0 < 5.0:
        frame = camera.value
        if frame is None: time.sleep(0.01); continue
        out = model(preprocess(frame))
        xy = out.detach().float().cpu().numpy().flatten()
        if abs(xy[0]) < 0.25: break
        time.sleep(0.01)
    robot.stop(); flush_camera(10)

# =========================================================
# 4. 작업 분배기
# =========================================================
def execute_work_logic(task_type, location):
    print(f"⚙️ [WORK] Executing '{task_type}' at '{location}'...")
    try:
        if task_type == "deliver_water":
            if location == "WATER": agv_hw.align_and_pick('cup') 
            elif location.startswith("RES_"): agv_hw._place_cup_motion()
        elif task_type == "collect_cup":
            if location.startswith("RES_"): agv_hw.align_and_pick('cup')
            elif location == "WATER": agv_hw._place_cup_motion()
        elif task_type == "collect_laundry":
            if location.startswith("RES_"): agv_hw.align_and_pick('laundry')
            elif location == "DROP": agv_hw._place_block()
    except Exception as e:
        print(f"⚠️ [WORK] Error: {e}")
        agv_hw.reset_servos()
    print(f"⚙️ [WORK] Finished.")

# =========================================================
# 5. Task Start
# =========================================================
def start_task(task_id, task_type, target_area_input):
    global line_follow_enabled, state, current_task_id, current_task_type
    global current_waypoints, target_idx, track_dir, target_area, angle_last, _last_log_t, force_arrival

    tgt_nm = str(target_area_input or "RES_B").upper().strip()
    typ = str(task_type or "unknown").lower().strip()
    current_task_type = typ 

    wps = []
    if typ == "deliver_water": wps = ["WATER", tgt_nm, "CHARGE"]
    elif typ == "collect_cup": wps = [tgt_nm, "WATER", "CHARGE"]
    elif typ == "collect_laundry": wps = [tgt_nm, "DROP", "CHARGE"]
    else: wps = [tgt_nm]

    valid_wps = [w for w in wps if w in AREA_TO_IDX]
    if not valid_wps: print(f"[TASK] Invalid wps: {wps}"); return

    with _lock:
        if line_follow_enabled: return
        first_dest = valid_wps.pop(0)
        tgt_idx = int(AREA_TO_IDX[first_dest])
        with track_lock: cur_idx = int(track_idx_f)
        current_waypoints = valid_wps
        desired_dir = choose_shortest_dir(cur_idx, tgt_idx)
        if desired_dir != track_dir:
            print(f"[TURN] Init Smart 180"); smart_turn_180("left")
            track_dir = desired_dir
        with state_lock:
            state = "running"; current_task_id = task_id
            target_area = first_dest; target_idx = tgt_idx
        angle_last = 0.0; _last_log_t = 0.0
        flush_camera(10)
        line_follow_enabled = True
    print(f"[TASK] START {typ} -> wps={wps}")

def stop_line_follow():
    global line_follow_enabled
    with _lock: line_follow_enabled = False
    try: robot.stop()
    except: pass
    print("[LINE] Force STOP.")

# =========================================================
# 6. 메인 루프 (DONE 수정됨)
# =========================================================
def _line_loop():
    global line_follow_enabled, state, done_until, target_area, target_idx, track_dir
    global current_waypoints, angle_last, _last_log_t, force_arrival

    print("[LOOP] Step 4 Final (Resource Optimized).")
    arrive_t0 = None
    
    _SPEED = globals().get('SPEED', 0.30)
    _GAIN = globals().get('STEERING_GAIN', 0.20)
    _DGAIN = globals().get('STEERING_DGAIN', 0.05)
    _BIAS = globals().get('STEERING_BIAS', 0.00)
    _DT = globals().get('LINE_LOOP_DT', 0.02)

    try:
        while not line_stop.is_set():
            with _lock: enabled = line_follow_enabled
            if not enabled:
                update_track_progress_if_running(False, False)
                time.sleep(0.02); continue

            # 1. 도착 판정
            is_arrived = False; dist_ok = False
            if state == "running" and target_idx is not None:
                with track_lock: cur_i = int(track_idx_f) % globals().get('TRACK_N', 800)
                d = ring_dist(cur_i, int(target_idx))
                if d <= _ARRIVE_IDX_TOL: dist_ok = True
            
            if force_arrival:
                is_arrived = True; force_arrival = False
            elif dist_ok:
                if arrive_t0 is None: arrive_t0 = time.monotonic()
                elif time.monotonic() - arrive_t0 >= _ARRIVE_HOLD_SEC: is_arrived = True
            else: arrive_t0 = None
            
            # 2. 도착 시
            if is_arrived:
                robot.stop(); time.sleep(0.15)
                snap_track_idx(int(target_idx))
                curr_dest_name = target_area
                print(f"[ARRIVE] Reached {curr_dest_name}")
                arrive_t0 = None

                with state_lock: state = "working"
                execute_work_logic(current_task_type, curr_dest_name)
                flush_camera(10) # ★★★ 여기서 CSI 카메라 버퍼 왕창 비움 (중요) ★★★

                if len(current_waypoints) > 0:
                    next_dest = current_waypoints.pop(0)
                    next_idx = int(AREA_TO_IDX[next_dest])
                    cur_i = int(target_idx)
                    desired = choose_shortest_dir(cur_i, next_idx)
                    if desired != track_dir:
                        print(f"[TURN] 180"); smart_turn_180("left")
                        track_dir = desired; angle_last = 0.0
                        flush_camera(10)
                    with state_lock:
                        state = "running"; target_area = next_dest; target_idx = next_idx
                    print(f"[NEXT] Heading to {next_dest}")
                    continue 
                else:
                    with state_lock:
                        state = "done"
                        target_area = None
                        done_until = time.monotonic() + _DONE_HOLD_SEC
                    with _lock: line_follow_enabled = False
                    print("[DONE] All tasks finished.")
                    continue

            # 3. 주행
            frame = camera.value
            if frame is None: 
                update_track_progress_if_running(state=="running", False)
                time.sleep(0.02); continue

            try:
                out = model(preprocess(frame))
                xy = out.detach().float().cpu().numpy().flatten()
                x = float(xy[0]); y = float((0.5 - xy[1]) / 2.0)
                angle = float(np.arctan2(x, y))
                pid = angle * _GAIN + (angle - angle_last) * _DGAIN
                angle_last = angle
                steering = pid + _BIAS
                left  = float(np.clip(_SPEED + steering, 0.0, 1.0))
                right = float(np.clip(_SPEED - steering, 0.0, 1.0))
                
                robot.left_motor.value = left; robot.right_motor.value = right
                update_track_progress_if_running(state=="running", True)
                
                now = time.time()
                if _PRINT_MOTOR and (now - _last_log_t >= _LOG_EVERY_SEC):
                    _last_log_t = now
                    print(f"[RUN] x={x:+.2f} L={left:.2f} R={right:.2f} tgt={target_area}")
            except Exception as e:
                robot.stop(); time.sleep(0.1)
            time.sleep(_DT)

    finally:
        try: robot.stop()
        except: pass
        print("[LOOP] Line loop stopped.")

if 'line_thread' in globals() and line_thread.is_alive():
    line_stop.set(); line_thread.join()

line_stop.clear()
line_thread = threading.Thread(target=_line_loop, daemon=True)
line_thread.start()
print("[OK] Step4 Final RESTARTED (Resource Optimized)")

Succeeded to open the port
Succeeded to change the baudrate
[LOOP] Step 4 Final (Resource Optimized).[OK] Step4 Final RESTARTED (Resource Optimized)



In [9]:
mqtt_connected = threading.Event()
TOPIC_DEV = "robot/agv1/dev"

def on_connect(c, userdata, flags, rc, properties=None):
    if rc == 0:
        c.subscribe(TOPIC_TASK, qos=0)
        c.subscribe(TOPIC_DEV, qos=0)
        print(f"[MQTT] connected & subscribed ({TOPIC_TASK}, {TOPIC_DEV})")
        mqtt_connected.set()

def on_message(c, userdata, msg):
    try:
        text = msg.payload.decode("utf-8", errors="ignore").strip()
        obj = json.loads(text) if text else {}
    except:
        return

    # 1. Dev/Cheat 명령
    if msg.topic == TOPIC_DEV:
        cmd = str(obj.get("cmd") or "").lower()
        if cmd == "fix_pose":
            area_key = str(obj.get("area") or "").upper()
            if area_key in AREA_TO_IDX:
                new_idx = int(AREA_TO_IDX[area_key])
                snap_track_idx(new_idx)
                print(f"[CHEAT] Force position sync to {area_key} (idx={new_idx})")
                
                with state_lock:
                    global area, target_area, force_arrival, state
                    area = area_key
                    if state == "running" and target_area == area_key:
                        force_arrival = True
                        print(f"[CHEAT] Target matched! Force Stopping...")
            else:
                print(f"[CHEAT] Unknown area code: {area_key}")
        return

    # 2. 일반 Task
    if isinstance(obj, dict):
        cmd = str(obj.get("cmd") or obj.get("action") or "").lower().strip()
        if cmd in ("stop", "off", "cancel", "idle", "done"):
            stop_line_follow()
            return

    task_id = obj.get("task_id") or obj.get("id") or f"t_{int(time.time())}"
    task_type = obj.get("type") or "unknown"
    target_area = obj.get("target_area") or obj.get("target")

    start_task(task_id, task_type, target_area)

# 기존 클라이언트 정리
if 'client' in globals():
    try: client.loop_stop(); client.disconnect()
    except: pass

client = mqtt.Client(client_id=CLIENT_ID)
client.on_connect = on_connect
client.on_message = on_message
client.reconnect_delay_set(min_delay=1, max_delay=5)

print(f"[MQTT] connecting to {BROKER_IP}:{BROKER_PORT} ...")
client.connect(BROKER_IP, BROKER_PORT, keepalive=60)
client.loop_start()

# ===== [함수 추가] CPU 온도 읽기 =====
def get_cpu_temp():
    try:
        with open("/sys/devices/virtual/thermal/thermal_zone0/temp", "r") as f:
            return round(float(f.read().strip()) / 1000.0, 1)
    except:
        return 0.0

# ===== [함수 추가] 배터리 읽기 (팀원 코드 로직 적용) =====
def get_battery_pct():
    if not ina: return 0
    try:
        v = ina.getBusVoltage_V()
        # 팀원 공식: (전압 / 12.6V) * 100
        pct = (v / 12.6) * 100
        return int(max(0, min(100, pct))) # 0~100 사이로 안전하게 자름
    except:
        return 0

def _status_loop():
    global state, current_task_id 
    print("[PUB] status publisher started")
    while True:
        mqtt_connected.wait()
        with state_lock:
            if state == "done" and time.monotonic() >= done_until:
                state = "idle"
                current_task_id = None
            st = state
            tid = current_task_id
            ar = area
            tar = target_area

        pose, dbg = get_track_pose()
        
        # [핵심] 실제 값 읽어오기
        real_cpu = get_cpu_temp()
        real_bat = get_battery_pct()
        
        # global battery 변수 업데이트 (혹시 다른데서 쓸까봐)
        global battery
        battery = real_bat

        payload = {
            "robot_id": robot_id,
            "state": st,
            "task_id": tid,
            "battery": real_bat,  # 진짜 배터리
            "cpu_temp": real_cpu, # 진짜 온도
            "area": ar,
            "target_area": tar,
            "pose": pose,
            "updated_at": int(time.time() * 1000),
            "_debug": dbg,
        }
        try: client.publish(TOPIC_STATUS, json.dumps(payload), qos=0)
        except: pass
        time.sleep(STATUS_PERIOD_SEC)

if '_status_thread' not in globals() or not _status_thread.is_alive():
    _status_thread = threading.Thread(target=_status_loop, daemon=True)
    _status_thread.start()

[MQTT] connecting to 10.41.145.221:1883 ...
[PUB] status publisher started
[MQTT] connected & subscribed (robot/agv1/task, robot/agv1/dev)
[TASK] START collect_laundry -> wps=['RES_A', 'DROP', 'CHARGE']
[RUN] x=-0.09 L=0.02 R=0.58 tgt=RES_A
[RUN] x=-0.02 L=0.26 R=0.34 tgt=RES_A
[RUN] x=-0.03 L=0.24 R=0.36 tgt=RES_A
[RUN] x=-0.05 L=0.22 R=0.38 tgt=RES_A
[RUN] x=-0.09 L=0.15 R=0.45 tgt=RES_A
[RUN] x=-0.23 L=0.08 R=0.52 tgt=RES_A
[RUN] x=-0.07 L=0.15 R=0.45 tgt=RES_A
[RUN] x=-0.10 L=0.04 R=0.56 tgt=RES_A
[RUN] x=-0.05 L=0.10 R=0.50 tgt=RES_A
[RUN] x=+0.00 L=0.34 R=0.26 tgt=RES_A
[RUN] x=+0.09 L=0.51 R=0.09 tgt=RES_A
[RUN] x=-0.04 L=0.19 R=0.41 tgt=RES_A
[RUN] x=-0.06 L=0.14 R=0.46 tgt=RES_A
[RUN] x=-0.00 L=0.30 R=0.30 tgt=RES_A
[RUN] x=-0.02 L=0.27 R=0.33 tgt=RES_A
[RUN] x=-0.02 L=0.26 R=0.34 tgt=RES_A
[RUN] x=-0.05 L=0.18 R=0.42 tgt=RES_A
[RUN] x=-0.02 L=0.22 R=0.38 tgt=RES_A
[RUN] x=-0.04 L=0.18 R=0.42 tgt=RES_A
[ARRIVE] Reached RES_A
⚙️ [WORK] Executing 'collect_laundry' at 'RES_A'...


In [ ]:
def shutdown():
    stop_line_follow()
    try: line_stop.set()
    except: pass
    time.sleep(0.1)
    try:
        client.loop_stop()
        client.disconnect()
    except: pass
    try: robot.stop()
    except: pass
    try: camera.stop()
    except: pass
    print("[DONE] shutdown")

# 필요할 때만 주석 풀고 실행하세요
shutdown()